
### This is the graph of fuel spend.  
The database work was done in __/media/lnr-ai/christo/github_repos/graphing/project_45500.cql__ 

In this work we aim to achieve the following:
Merchant rank each of the fuel station communities where the member count is greater than 4.

Load libraries:

In [70]:
import pandas as pd
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
import time
import glob
import os

### Merchant ranking

The following needs to run only once. It creates a parameter  called __merchantRank_45500__ on each of the fuel station merchant nodes that belongs to a community of fuel stations where the member counts is more than some minimum.  

This portion of the script only needs to run only once

Define neo4j query to extract all fuel station communities as created in /media/lnr-ai/christo/github_repos/graphing/project_45500.cql.  Draw the community number and the member count:

In [71]:
neo4j_query="""
MATCH (m:Merchant {subclass_id:45500})
RETURN COUNT(m.merchant_45500_community) as membercount, m.merchant_45500_community as merchant_45500_community
ORDER BY membercount ASC;
"""
merchant_rank_df=graph.run(neo4j_query).to_data_frame()

View this dataframe:

In [72]:
merchant_rank_df

,membercount,merchant_45500_community
0,1,0
1,1,11373
2,1,4
3,1,11374
4,1,5
...,...,...
8155,205,1938
8156,227,4018
8157,237,2776
8158,268,6821


Limit this dataframe to communities with more than 4 members:

In [73]:
merchant_rank_df=merchant_rank_df[merchant_rank_df['membercount']>4].copy()

Create a unique list of all community members:

In [74]:
merchant_45500_community_list=list(set(merchant_rank_df['merchant_45500_community']))

Save the above dataframe to a .csv file:

In [75]:
merchant_rank_df_copy=merchant_rank_df.copy()
merchant_rank_df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_community_df',index=False)

Set the first element of merchant_45500_community_list equal to merchant_45500_community:

In [76]:
merchant_45500_community=merchant_45500_community_list[0]

Create a subgraph of all merchants in the merchant_45500_community; we will drop this graph in the iteration that follows.

In [77]:
# Create a subgraph of all merchants in merchant_45500_community:
print('Create subgraph on community {merchant_45500_community}:'.format(merchant_45500_community=merchant_45500_community))
neo4j_query="""CALL gds.graph.create.cypher(
    'merchant_45500_graph',
    'MATCH (m0:Merchant {{subclass_id:45500}}) 
    WHERE m0.merchant_45500_community = {merchant_45500_community} RETURN id(m0) AS id, 
    m0.totaltransactionamount as totaltransactionamount, m0.totaltransactioncount as totaltransactioncount',
    'MATCH (m0:Merchant)-[rel:MERCHANT_FEET_LINK_45500]->(m1:Merchant) 
    WHERE rel.count > 1 AND 
    m0.merchant_45500_community = {merchant_45500_community} AND 
    m1.merchant_45500_community = {merchant_45500_community}
    RETURN id(m0) AS source, id(m1) AS target, rel.count as count'
)
YIELD graphName, nodeCount, relationshipCount, createMillis;
""".format(merchant_45500_community=merchant_45500_community)
graph.run(neo4j_query).data()


Create subgraph on community 5638:


[{'graphName': 'merchant_45500_graph',
  'nodeCount': 15,
  'relationshipCount': 52,
  'createMillis': 1581}]

In the following iteration we iterate over the contents of the merchant_45500_community_list list.

In [ ]:
merchant_rank_df=pd.DataFrame()
# merchant_45500_community=merchant_45500_community_list[0]
A=merchant_45500_community_list.copy()
counter=0
for merchant_45500_community in merchant_45500_community_list:
    counter+=1
    print('Doing community {merchant_45500_community}; {counter} of {countlen}'.format(merchant_45500_community=merchant_45500_community, counter=counter,countlen=len(merchant_45500_community_list)))
    print('Drop subgraph:')
    neo4j_query="""CALL gds.graph.drop('merchant_45500_graph') YIELD graphName;"""
    graph.run(neo4j_query).data()
# Create a subgraph of all merchants in merchant_45500_community:
    print('Create subgraph on community {merchant_45500_community}:'.format(merchant_45500_community=merchant_45500_community))
    neo4j_query="""CALL gds.graph.create.cypher(
        'merchant_45500_graph',
        'MATCH (m0:Merchant {{subclass_id:45500}}) 
        WHERE m0.merchant_45500_community = {merchant_45500_community} RETURN id(m0) AS id, 
        m0.totaltransactionamount as totaltransactionamount, m0.totaltransactioncount as totaltransactioncount',
        'MATCH (m0:Merchant)-[rel:MERCHANT_FEET_LINK_45500]->(m1:Merchant) 
        WHERE rel.count > 1 AND 
        m0.merchant_45500_community = {merchant_45500_community} AND 
        m1.merchant_45500_community = {merchant_45500_community}
        RETURN id(m0) AS source, id(m1) AS target, rel.count as count'
    )
    YIELD graphName, nodeCount, relationshipCount, createMillis;
    """.format(merchant_45500_community=merchant_45500_community)
    graph.run(neo4j_query).data()
# Write the MerchantRank on community nodes.  The rank belongs to the community of merchants as defined by the sub graph created above:
    print('MerchantRank community members:')
    neo4j_query="""
    CALL gds.pageRank.write('merchant_45500_graph', {
      maxIterations: 20,
      dampingFactor: 0.85,
      writeProperty: 'merchantRank_45500',
      relationshipWeightProperty: 'count'
    })
    YIELD nodePropertiesWritten AS writtenProperties, ranIterations;"""
    graph.run(neo4j_query).data()
    print('Extract community members with merchant rank: ')
    neo4j_query="""
    MATCH (m:Merchant {{subclass_id:45500}}) WHERE m.merchant_45500_community = {merchant_45500_community}
    RETURN m.franchisename as franchisename, m.companyname as companyname,  m.merchantRank_45500 as MerchantRank, {merchant_45500_community} as merchant_45500_community 
    ORDER BY MerchantRank DESC;
    """.format(merchant_45500_community=merchant_45500_community)
    df=graph.run(neo4j_query).to_data_frame()

    df['rankID']=range(1,df.shape[0]+1)
    print('Append extracted merchant community to a result dataframe:')    
    merchant_rank_df=merchant_rank_df.append(df,ignore_index=True)
    A.pop(A.index(merchant_45500_community))
    print('====================================================================================================')

In [ ]:
A

In [ ]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')

In [ ]:
# merchant_rank_df=pd.DataFrame()
merchant_45500_community_list=A.copy()
counter=0
for merchant_45500_community in merchant_45500_community_list:
    counter+=1
    print('Doing community {merchant_45500_community}; {counter} of {countlen}'.format(merchant_45500_community=merchant_45500_community, counter=counter,countlen=len(merchant_45500_community_list)))
    print('Drop subgraph:')
    neo4j_query="""CALL gds.graph.drop('merchant_45500_graph') YIELD graphName;"""
    graph.run(neo4j_query).data()
# Create a subgraph of all merchants in merchant_45500_community:
    print('Create subgraph on community {merchant_45500_community}:'.format(merchant_45500_community=merchant_45500_community))
    neo4j_query="""CALL gds.graph.create.cypher(
        'merchant_45500_graph',
        'MATCH (m0:Merchant {{subclass_id:45500}}) 
        WHERE m0.merchant_45500_community = {merchant_45500_community} RETURN id(m0) AS id, 
        m0.totaltransactionamount as totaltransactionamount, m0.totaltransactioncount as totaltransactioncount',
        'MATCH (m0:Merchant)-[rel:MERCHANT_FEET_LINK_45500]->(m1:Merchant) 
        WHERE rel.count > 1 AND 
        m0.merchant_45500_community = {merchant_45500_community} AND 
        m1.merchant_45500_community = {merchant_45500_community}
        RETURN id(m0) AS source, id(m1) AS target, rel.count as count'
    )
    YIELD graphName, nodeCount, relationshipCount, createMillis;
    """.format(merchant_45500_community=merchant_45500_community)
    graph.run(neo4j_query).data()
# Write the MerchantRank on community nodes.  The rank belongs to the community of merchants as defined by the sub graph created above:
    print('MerchantRank community members:')
    neo4j_query="""
    CALL gds.pageRank.write('merchant_45500_graph', {
      maxIterations: 20,
      dampingFactor: 0.85,
      writeProperty: 'merchantRank_45500',
      relationshipWeightProperty: 'count'
    })
    YIELD nodePropertiesWritten AS writtenProperties, ranIterations;"""
    graph.run(neo4j_query).data()
    print('Extract community members with merchant rank: ')
    neo4j_query="""
    MATCH (m:Merchant {{subclass_id:45500}}) WHERE m.merchant_45500_community = {merchant_45500_community}
    RETURN m.franchisename as franchisename, m.companyname as companyname,  m.merchantRank_45500 as MerchantRank, {merchant_45500_community} as merchant_45500_community 
    ORDER BY MerchantRank DESC;
    """.format(merchant_45500_community=merchant_45500_community)
    df=graph.run(neo4j_query).to_data_frame()

    df['rankID']=range(1,df.shape[0]+1)
    print('Append extracted merchant community to a result dataframe:')    
    merchant_rank_df=merchant_rank_df.append(df,ignore_index=True)
    A.pop(A.index(merchant_45500_community))    
    print('====================================================================================================')

Save the contents of merchant_rank_df into a .csv file:

In [ ]:
merchant_rank_df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_rank_df',index=False)

We now have two dataframes thus far.  Dataframe merchant_45500_community_df ontains only the community number and the community count:

In [82]:
merchant_45500_community_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_community_df')

In [80]:
list(merchant_45500_community_df)

['membercount', 'merchant_45500_community']

Dataframe merchant_rank_df contains the community number (merchant_45500_community) and the rank of the merchant (MerchantRank) in that community.  It also has the franchise and company names. Read the same file into merchant_rank_df:

In [92]:
merchant_45500_rank_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_rank_df')

In [93]:
list(merchant_45500_rank_df)

['franchisename',
 'companyname',
 'MerchantRank',
 'merchant_45500_community',
 'rankID']

Merge merchant_45500_rank_df and merchant_45500_community_df on the merchant_45500_community column:

In [ ]:
df=merchant_45500_rank_df.merge(merchant_45500_community_df,left_on='merchant_45500_community',
                             right_on='merchant_45500_community',how='left')

In [15]:
df.to_csv('/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_df',index=False)

In [14]:
len(set(df['merchant_45500_community']))

78

### Community aggregation:

Here we use the results of the code above to do aggregation on the fuel station communities:

Connect to neo4J:

In [38]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')

In what follows we do two pieces of aggregation
1. By merchant. Include 
    1. community number (merchant_45500_community),
    2. companyname,
    3. franchisename,
    4. merchant rank, 
    5. total transaction amount and 
    6. total number of transactions.    

In [39]:
neo4j_query="""MATCH (merchant:Merchant {subclass_id:45500})
WHERE EXISTS(merchant.merchantRank_45500)
RETURN merchant.merchant_45500_community as merchant_45500_community,
merchant.companyname as companyname, 
merchant.franchisename as franchisename, 
merchant.merchantRank_45500 as merchant_rank,
merchant.totaltransactionamount as totaltransactionamount,
merchant.totaltransactioncount as totaltransactioncount
ORDER BY merchant_45500_community ASC;
"""
# merchant_rank_df=merchant_rank_df.append(df,ignore_index=True)
df=graph.run(neo4j_query).to_data_frame()

Create a new column __franchise_rank__.  This column holds the rank of a merchant in a merchant community based on the merchant rank, 1 being the franchise with the highest merchant rank - the franchise most likely to visited by a shopper.

In [40]:
df['franchise_rank']=df.groupby("merchant_45500_community")["merchant_rank"].rank("dense", ascending=False)

Look at one company and ask the question of how many merchants see this company with the highest merchant rank:

In [44]:
df[(df.companyname=='BP')&(df.franchise_rank==1)]

,merchant_45500_community,companyname,franchisename,merchant_rank,totaltransactionamount,totaltransactioncount,franchise_rank
9,358,BP,BP KIASHA PA,4.283081,-722820.14,2992,1.0
1570,2971,BP,BP ELDORADO,1.259391,-206941.48,1122,1.0
2342,3605,BP,BP VALLEY MO,2.034436,-581875.97,1729,1.0


In [45]:
len(set(df.merchant_45500_community))

78

In [47]:
df[(df.franchise_rank==1)]

,merchant_45500_community,companyname,franchisename,merchant_rank,totaltransactionamount,totaltransactioncount,franchise_rank
3,346,ENGEN,ENGEN PANDA,1.975369,-197556.70,947,1.0
9,358,BP,BP KIASHA PA,4.283081,-722820.14,2992,1.0
36,478,ENGEN,ENGEN HI Q A,2.466796,-368401.60,1326,1.0
54,686,SHELL,SHELL NONGOM,2.658348,-518676.51,1590,1.0
67,975,ENGEN,ENGEN BULT,3.486104,-306677.61,1950,1.0
...,...,...,...,...,...,...,...
3720,6438,TOTAL,TOTAL RA DES,3.975671,-538831.30,3008,1.0
3860,6745,SHELL,SHELL LADYSM,3.999127,-713923.88,2570,1.0
3993,6821,SHELL,SHELL NOORDW,9.255524,-686646.90,3122,1.0
4145,7697,CALTEX,CALTEX UITSIG,0.624765,-75689.36,472,1.0


In [50]:
df[(df.merchant_45500_community==11222)]

,merchant_45500_community,companyname,franchisename,merchant_rank,totaltransactionamount,totaltransactioncount,franchise_rank
4146,11222,SASOL,SASOL WALKERVI,1.791241,-441423.42,1708,1.0
4147,11222,ENGEN,ENGEN WALKER,0.291602,-84630.80,244,5.0
4148,11222,ENGEN,ENGEN WALKERVI,0.959515,-20953.05,60,4.0
4149,11222,SHELL,SHELL SOUTH CI,0.150000,-80809.00,292,6.0
4150,11222,TOTAL,TOTAL DE DEUR,1.776675,-49356.46,211,2.0
4151,11222,SASOL,SASOL WALKERVILLE,0.150000,-60230.52,142,6.0
4152,11222,TOTAL,TOTAL DE DEU,1.650348,-177508.54,834,3.0


Add another two columns, __totaltransactionamount_rank__ is the rank of all franchises in a community base on __totaltransactionamount__ and __totaltransactioncount_rank__ is likewise based on __totaltransactioncount__.

In [53]:
# df['totaltransactionamount']=-df['totaltransactionamount']
df['totaltransactionamount_rank']=df.groupby("merchant_45500_community")["totaltransactionamount"].rank("dense", ascending=False)
df['totaltransactioncount_rank']=df.groupby("merchant_45500_community")["totaltransactioncount"].rank("dense", ascending=False)

In [56]:
df[['merchant_45500_community','totaltransactionamount','totaltransactioncount','franchise_rank','totaltransactionamount_rank','totaltransactioncount_rank']]

,merchant_45500_community,totaltransactionamount,totaltransactioncount,franchise_rank,totaltransactionamount_rank,totaltransactioncount_rank
0,346,18553.60,78,6.0,7.0,7.0
1,346,292259.29,1135,5.0,3.0,3.0
2,346,427067.14,1508,3.0,1.0,2.0
3,346,197556.70,947,1.0,4.0,4.0
4,346,159964.26,516,4.0,5.0,5.0
...,...,...,...,...,...,...
4148,11222,20953.05,60,4.0,7.0,7.0
4149,11222,80809.00,292,6.0,4.0,3.0
4150,11222,49356.46,211,2.0,6.0,5.0
4151,11222,60230.52,142,6.0,5.0,6.0


In [55]:
df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_community_aggregation_df',index=False)

#### Total client spend by fuel station clients:

In the following aggregation we take all clients who spent at any off the merchant_45500_community fuel stations and aggregate their total spend (at all POS).  We also count the number of distinct clients and merchants belonging to each community.

In [85]:
neo4j_query="""
MATCH (client:Client)-[t:TRANSACTED_AT]-(merchant:Merchant)
WHERE EXISTS(merchant.merchantRank_45500)
RETURN merchant.merchant_45500_community as merchant_45500_community,
COUNT(DISTINCT client) as nClients,
SUM(client.totaltransactionamount) as totalspend,
COUNT(DISTINCT merchant) as nMerchants
ORDER BY nClients ASC;
"""
df=graph.run(neo4j_query).to_data_frame()

In [86]:
df

,merchant_45500_community,nClients,totalspend,nMerchants
0,4028,729,-2.347251e+07,5
1,2065,960,-3.066793e+07,5
2,5606,1030,-2.524022e+07,7
3,2126,1085,-3.262959e+07,6
4,3817,1118,-3.409385e+07,8
...,...,...,...,...
73,1938,60110,-3.164864e+09,205
74,4018,63462,-2.854820e+09,227
75,2776,71139,-3.219391e+09,237
76,3005,80820,-4.073124e+09,320


In [89]:
min(df['totalspend']/df['nClients'])

-52896.249659140805

Save the results to merchant_45500_community_spend_df:

In [59]:
df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_community_spend_df',index=False)

Read the previous two files:

In [95]:
community_aggregation_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_community_aggregation_df')
community_spend_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_45500_community_spend_df')

In [96]:
list(community_aggregation_df)

['merchant_45500_community',
 'companyname',
 'franchisename',
 'merchant_rank',
 'totaltransactionamount',
 'totaltransactioncount',
 'franchise_rank',
 'totaltransactionamount_rank',
 'totaltransactioncount_rank']

In [97]:
list(community_spend_df)

['merchant_45500_community', 'nClients', 'totalspend', 'nMerchants']

In [98]:
community_aggregation_df.merge(community_spend_df,how='left',left_on='merchant_45500_community',right_on='merchant_45500_community')

,merchant_45500_community,companyname,franchisename,merchant_rank,totaltransactionamount,totaltransactioncount,franchise_rank,totaltransactionamount_rank,totaltransactioncount_rank,nClients,totalspend,nMerchants
0,346,ENGEN,ENGEN PANDA MOTORS,0.150000,18553.60,78,6.0,7.0,7.0,3090,-68255088.24,7
1,346,SHELL,SHELL SUMMER,0.424530,292259.29,1135,5.0,3.0,3.0,3090,-68255088.24,7
2,346,CALTEX,Cltx Le Mans,1.456385,427067.14,1508,3.0,1.0,2.0,3090,-68255088.24,7
3,346,ENGEN,ENGEN PANDA,1.975369,197556.70,947,1.0,4.0,4.0,3090,-68255088.24,7
4,346,TOTAL,TOTAL WINGATE,0.750491,159964.26,516,4.0,5.0,5.0,3090,-68255088.24,7
...,...,...,...,...,...,...,...,...,...,...,...,...
4148,11222,ENGEN,ENGEN WALKERVI,0.959515,20953.05,60,4.0,7.0,7.0,1693,-40370950.30,7
4149,11222,SHELL,SHELL SOUTH CI,0.150000,80809.00,292,6.0,4.0,3.0,1693,-40370950.30,7
4150,11222,TOTAL,TOTAL DE DEUR,1.776675,49356.46,211,2.0,6.0,5.0,1693,-40370950.30,7
4151,11222,SASOL,SASOL WALKERVILLE,0.150000,60230.52,142,6.0,5.0,6.0,1693,-40370950.30,7


Here we look at totals by community and create new columns __community_totaltransactionamount__ and __community_totaltransactioncount__

In [90]:
neo4j_query="""MATCH (merchant:Merchant {subclass_id:45500})
WHERE EXISTS(merchant.merchantRank_45500)
RETURN merchant.merchant_45500_community as merchant_45500_community,
COUNT(merchant.merchant_45500_community) as count_merchant_45500_community,
SUM(merchant.totaltransactionamount) as community_totaltransactionamount,
SUM(merchant.totaltransactioncount) as community_totaltransactioncount
ORDER BY merchant_45500_community ASC;
"""
df=graph.run(neo4j_query).to_data_frame()

In [91]:
df

,merchant_45500_community,count_merchant_45500_community,community_totaltransactionamount,community_totaltransactioncount
0,346,7,-1434187.51,6712
1,358,27,-8085643.23,35864
2,478,15,-3503355.64,12877
3,686,16,-4407264.20,15353
4,975,25,-3517133.49,17158
...,...,...,...,...
73,6438,140,-26447042.09,128154
74,6745,22,-4526567.10,16871
75,6821,268,-69571232.32,285461
76,7697,7,-1015858.60,3287
